In [8]:
from dotenv import load_dotenv
load_dotenv()

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
import os
import weaviate
from weaviate.classes.init import Auth

wcd_url = os.environ["WCD_URL"]
wcd_api_key = os.environ["WCD_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,                                    
    auth_credentials=Auth.api_key(wcd_api_key),
)

/Users/sgarrity/projects/chainlit_demo/.venv/lib/python3.13/site-packages/weaviate/warnings.py:329: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [9]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-large")

Settings.embed_model = embed_model

embeddings = embed_model.get_text_embedding(
    "OpenAI new Embeddings models is great."
)

print(embeddings[:5])

print("\n")

print(len(embeddings))

[-0.007739675231277943, 0.03943515568971634, -0.017037110403180122, -0.019970186054706573, 0.024573294445872307]


3072


In [10]:
# Ingest the docs in the data folder

docs = SimpleDirectoryReader('./data').load_data()

parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(docs)

/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/selector_events.py:869: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=79 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


In [11]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore


# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="Steve")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context)

In [13]:

query_engine = index.as_query_engine()
response = query_engine.query("reasons that my tesla's velocity is lower than usual")
print(response)

Your Tesla's velocity may be lower than usual due to a reduction in the vehicle's top speed and slower response to acceleration requests.
